# CNNと表を組み合わせて考えられないか？

MobileNetとニューラルネットワークを最後の層で結合することで問題の解答を実現する試み

In [ ]:
!pip install timm

In [ ]:
import math
import random
import time
import warnings
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers as T
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import pandas as pd
import torchvision.models as models
import timm

def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed = 2021
seed_torch(seed)

In [ ]:
train_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')


## Modelの構築
ResNetを基礎として利用

In [ ]:
class Create_Model(nn.Module):
    def __init__(self):
        super(Create_Model,self).__init__()
        self.swin = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True,num_classes=0, in_chans=3)
        num_features = self.swin.num_features
        self.fc = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features, 1)
        )
    def forward(self,x,xx):
        x = self.swin(x)
        x = self.fc(x)
        return x

In [ ]:
import torch.nn.functional as F
import torchvision.models as models
class Pawpularity_Net(nn.Module):
    def __init__(self):
        super(Pawpularity_Net , self).__init__()
        self.model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True,num_classes=0, in_chans=3)
        num_features = self.model.num_features
        #self.model   = nn.Sequential(*list(model.children())[:-1])
        #self.layer_1 = nn.Linear(12,100)
        #self.bn1     = nn.BatchNorm1d(100)
        #self.layer_2 = nn.Linear(100,100)
        #self.bn2     = nn.BatchNorm1d(100)
        #self.layer_3 = nn.Linear(2148 , 500)
        #self.bn3     = nn.BatchNorm1d(500)
        self.last_layer = nn.Linear(num_features, 1)
    
    def forward(self,x1,x2):
        x1 = self.model(x1)
        #x1 = x1.view(x1.shape[0],-1)
        #x2 = F.relu(self.layer_1(x2))
        #x2 = F.relu(self.layer_2(x2))
        #x = torch.cat([x1,x2],dim = 1)
        #x = F.relu(self.layer_3(x))
        #x = self.last_layer(x)
        x = self.last_layer(F.relu(x1))
        
        return x

In [ ]:
from torch.utils.data import DataLoader, Dataset
from PIL import Image 
import torchvision.transforms as T
from tqdm import tqdm

def get_transforms(key):
    transforms = {'train':T.Compose([
                            T.RandomHorizontalFlip(),
                            T.RandomVerticalFlip(),
                            T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                            T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                            T.Resize(256),
                            T.CenterCrop(224),
                            T.ToTensor(),
                            T.Normalize(
                                mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
                            ]),
                           'val':T.Compose([
                            T.Resize(256),
                            T.CenterCrop(224),
                            T.ToTensor(),
                            T.Normalize(
                                mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
                            ])}
    
    return transforms[key]



        
class CustomDataSet(Dataset):
    def __init__(self,df,img_pth,is_test=False,now = 'val'):
        self.img_list =  torch.load(img_pth)
        self.table_list = df[df.columns[1:13]]
        self.label_list = df[df.columns[-1]].to_list()
        self.preprocess = get_transforms(now)
        self.is_test = is_test
        

            
    def __getitem__(self,idx):
        
        #if self.is_test is not True:
        #    path = "/kaggle/input/petfinder-pawpularity-score/train/"+self.img_list[idx]+'.jpg'
        #else:
        #    path = "/kaggle/input/petfinder-pawpularity-score/test/"+self.img_list[idx]+'.jpg'
        t1 = time.time()
        #img = Image.open(path).convert('RGB')
        img = self.img_list[idx]
        t2 = time.time()
        #print(f'読み出し:{t2 - t1}')
        read_time = t2 - t1
        t1 = time.time()
        
        #img = self.preprocess(img)
        t2 = time.time()
        
        
        #print(f'変換:{t2 - t1}')
        trans_time = t2 - t1
        
        t1 = time.time()
        tbl = torch.tensor(list(self.table_list.iloc[idx])).to(torch.float32)
        t2 = time.time()
        
        tbl_time = t2 - t1
        
        #print(f'テーブル変換:{t2 - t1}')
        t1 = time.time()
        label = torch.tensor(self.label_list[idx]/100).to(torch.float32)
        t2 = time.time()
        #print(f'ラベル:{t2 - t1}')
        label_time = t2 - t1
        
        sum_time = read_time + trans_time + tbl_time + label_time
        
        #print(f'画像読み出し：{read_time/sum_time*100} 画像変換:{trans_time/sum_time*100} テーブル変換:{tbl_time/sum_time*100} ラベル変換:{label_time/sum_time*100} 合計時間:{sum_time}')
        
        if self.is_test is not True:
            return img,tbl,label
        else:
            return img,tbl
    def __len__(self):
        return len(self.img_list)

    def __del__(self):
        del self.img_list 
        del self.table_list
        del self.label_list
        del self.preprocess
        del self.is_test 
    

In [ ]:
def get_img_list(img_list,types='train'):
    preprocess = get_transforms('val')
    print('make dataset...')
    tensor_block = [preprocess(Image.open(f"/kaggle/input/petfinder-pawpularity-score/{types}/"+img_list[i]+'.jpg').convert('RGB')) for i in range(len(img_list))]
    
    return tensor_block
    
            

In [ ]:
FOLD_NUM = 5

## 損失関数をMSELossからnn.BCEWithLogitsLossに変更
最強らしい

In [ ]:

from sklearn.model_selection import KFold
import torch.optim as opt
import numpy as np
import time
import gc

kf = KFold(n_splits=FOLD_NUM, random_state=None, shuffle=False)

BATCH_SIZE = 96
EPOCH = 50
DEVICE = 'cuda'
STOP_NUM = 3
criterion_mse = nn.MSELoss()
criterion_bce = nn.BCEWithLogitsLoss()

val_dataset = None
train_dataset = None
train_loader = None
val_loader = None
del train_dataset
del val_dataset
del train_loader
del val_loader

gc.collect()

for k,(train_index, test_index) in enumerate(kf.split(train_df['Id'])):
    print(f"========={k+1}-FOLD=========")
    model = Create_Model().to(DEVICE)
    val_dataset = None
    train_dataset = None
    train_loader = None
    val_loader = None
    
    train_dataset = CustomDataSet(train_df.loc[train_index],f"../input/petfinder-fold-dataset/{k+1}_fold_train.pth")
    val_dataset = CustomDataSet(train_df.loc[test_index],f"../input/petfinder-fold-dataset/{k+1}_fold_val.pth")
    train_loader = DataLoader(train_dataset,batch_size = BATCH_SIZE,shuffle = True,num_workers = 8)
    val_loader = DataLoader(val_dataset,batch_size = BATCH_SIZE,shuffle = False,num_workers = 8)
   

    
    optimizer = opt.AdamW(params = model.parameters(),lr = 1e-5)
    scheduler = opt.lr_scheduler.CosineAnnealingWarmRestarts(optimizer , 100 , eta_min=1e-4)
    best_loss = 99999
    stop_counter = 0
    
    for e in range(EPOCH):
        model.train()
        train_total_loss = 0
        train_score = 0
        counter = 0
        
        for n,(img,tbl,label) in enumerate(train_loader):
            img = img.to(DEVICE)
            tbl = tbl.to(DEVICE)
            label = label.to(DEVICE)
            optimizer.zero_grad()
            
            output = model(img,tbl)[:,0]
            
            loss = criterion_bce(output , label)
            score = criterion_mse(F.sigmoid(output)*100 , label*100).detach()
            
            
            train_total_loss = (train_total_loss * n + loss.detach().item())/(n+1)
            train_score =  (train_score * counter + score.item()*output.shape[0])/(counter + output.shape[0])
            
            counter += output.shape[0]
            loss.backward()
            optimizer.step()
            print('\rTRAIN EPOCH[{:03}/{:03}] ITR[{:04}/{:04}] LOSS:{:.5} SCORE:{:.5}'.format(e+1,EPOCH,n+1,len(train_loader),train_total_loss,np.sqrt(train_score)),end = "")
        scheduler.step()
        print()
        val_total_loss = 0
        val_score = 0
        counter = 0
        model.eval()
        with torch.no_grad():
            for n,(img,tbl,label) in enumerate(val_loader):
                img = img.to(DEVICE)
                tbl = tbl.to(DEVICE)
                label = label.to(DEVICE)
            
                output = model(img,tbl)[:,0]

                loss = criterion_bce(output , label)
                score = criterion_mse(F.sigmoid(output)*100 , label*100).detach()
                
                
                val_total_loss = (val_total_loss * n + loss.item())/(n+1)
                
                val_score =  (val_score * counter + score.item()*output.shape[0])/(counter + output.shape[0])
                
                counter += output.shape[0]
                print('\rVAL   EPOCH[{:03}/{:03}] ITR[{:04}/{:04}] LOSS:{:.5} SCORE:{:.5}'.format(e+1,EPOCH,n+1,len(val_loader),val_total_loss,np.sqrt(val_score)),end = "")
        print()
        if(best_loss > val_total_loss):
            best_loss = val_total_loss
            model_path = f'{k+1}-fold.pth'
            torch.save(model.state_dict(), model_path)
            stop_counter = 0
        else:
            stop_counter += 1
        if stop_counter >= STOP_NUM:
            break
    
    del train_dataset
    del val_dataset
    del train_loader
    del val_loader
    
    gc.collect()

In [ ]:
import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))
gc.collect()

In [ ]:
train_dataset = None
val_dataset = None

In [ ]:
"""
test_dataset = CustomDataSet(test_df,is_test = True)
test_loader = DataLoader(test_dataset,batch_size = 2,shuffle = False)
with torch.no_grad():
    fold_output = [[] for x in range(FOLD_NUM)]
    for k in range(FOLD_NUM):
        model = Pawpularity_Net().to(DEVICE)
        model_path = f'{k+1}-fold.pth'
        model.load_state_dict(torch.load(model_path))
        for n,(img,tbl) in enumerate(test_loader):
            img = img.to(DEVICE)
            tbl = tbl.to(DEVICE)
            output = model(img,tbl).cpu()[:,0]
            fold_output[k].append(F.sigmoid(output)*100)
            print('\r TEST FOLD[{:02}/{:02}]  ITR[{:03}/{:03}]'.format(k+1,FOLD_NUM,n+1,len(test_loader)),end = "")
"""

In [ ]:
"""
for k in range(len(fold_output)):
    fold_output[k] = torch.cat(fold_output[k],dim = 0).numpy()
"""

In [ ]:
#list(np.mean(np.array(fold_output),axis = 0))

In [ ]:
#df_result = pd.DataFrame({f'Pawpularity_{i}':fold_output[i] for i in range(len(fold_output))})

In [ ]:
#df_result

In [ ]:
#df_sub = pd.concat([train_df['Id'] , df_result],axis=1)

In [ ]:
#df_sub.to_csv('submission.csv', index=False)

## Trainデータに対する予測結果を出力

In [ ]:
"""
from sklearn.model_selection import KFold
import torch.optim as opt
import numpy as np


kf = KFold(n_splits=FOLD_NUM, random_state=None, shuffle=False)

BATCH_SIZE = 256
EPOCH = 5
DEVICE = 'cuda'


test_dataset = CustomDataSet(train_df)
test_loader = DataLoader(test_dataset,batch_size = BATCH_SIZE,shuffle = False,num_workers=2)
with torch.no_grad():
    fold_output = [[] for x in range(FOLD_NUM)]
    for k in range(FOLD_NUM):
        model = Pawpularity_Net().to(DEVICE)
        model_path = f'../input/bceleaningresult/{k+1}-fold.pth'
        model.load_state_dict(torch.load(model_path))
        for n,(img,tbl,label) in enumerate(test_loader):
            img = img.to(DEVICE)
            tbl = tbl.to(DEVICE)
            output = model(img,tbl).cpu()[:,0]
            fold_output[k].append(F.sigmoid(output)*100)
            print('\r TEST FOLD[{:02}/{:02}]  ITR[{:03}/{:03}]'.format(k+1,FOLD_NUM,n+1,len(test_loader)),end = "")
"""

## DatasetLoaderの速度確認

In [ ]:
"""
from sklearn.model_selection import KFold
import torch.optim as opt
import numpy as np
from tqdm import tqdm
import time

FOLD_NUM = 5
BATCH_SIZE = 32
EPOCH = 5
DEVICE = 'cuda'
kf = KFold(n_splits=FOLD_NUM, random_state=None, shuffle=False)


criterion_mse = nn.MSELoss(reduction = 'mean' )
criterion_bce = nn.BCEWithLogitsLoss()

for k,(train_index, test_index) in enumerate(kf.split(train_df['Id'])):
    train_dataset = CustomDataSet(train_df.loc[train_index])
    train_loader = DataLoader(train_dataset,batch_size = BATCH_SIZE,shuffle = True,num_workers=2)
    print(f"========={k}-FOLD=========len:{len(train_loader)}")
    print(f"Datanum:{BATCH_SIZE*len(train_loader)}")
    start = time.time()
    for n,(img,tbl,label) in enumerate(tqdm(train_loader)):
        pass
    end = time.time()
    print(end -start)
"""

In [ ]:
#torch.load("../input/petfinder-fold-dataset/1_fold_train.pth")